In [11]:
import pandas as pd
import numpy as np
import cv2
import os

In [12]:
POSE_CONNECTIONS = frozenset({
                                (0, 1), (0, 4), (1, 2), (2, 3), (3, 7), (4, 5), (5, 6), (6, 8), (9, 10), (11, 12),
                                (11, 13), (11, 23), (12, 14), (12, 24), (13, 15), (14, 16), (15, 17), (15, 19),
                                (15, 21), (16, 18), (16, 20), (16, 22), (17, 19), (18, 20), (23, 24), (23, 25),
                                (24, 26), (25, 27), (26, 28), (27, 29), (27, 31), (28, 30), (28, 32), (29, 31),
                                (30, 32)  
                            })

SELECTED_POINTS = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]

ANGLES = [[14, 12, 24], [12, 14, 16], [13, 11, 23], [11, 13, 15],
          [23, 24, 26], [24, 23, 25], [24, 26, 28], [23, 25, 27],
          [12, 24, 26], [11, 23, 25]]

In [13]:
def angle_between_lines(x1, y1, z1, x2, y2, z2, x4, y4, z4):
    vector1 = np.array([x1 - x2, y1 - y2, z1 - z2])
    vector2 = np.array([x4 - x2, y4 - y2, z4 - z2])
    
    vector1_norm = vector1 / np.linalg.norm(vector1)
    vector2_norm = vector2 / np.linalg.norm(vector2)
    
    cosine_angle = np.dot(vector1_norm, vector2_norm)
    
    angle_radians = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees / 360

In [14]:
def list_files_in_directory(directory):

    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if os.path.isfile(file_path):
                file_paths.append(file_path)
    return file_paths

In [15]:
directory = "aug_output"

if not os.path.exists(directory):
    os.makedirs(directory)

file_paths = list_files_in_directory("output")
len(file_paths)

3

In [16]:
for path in file_paths:

    df = pd.read_csv(path)
    df = df.drop("Unnamed: 0", axis = 1)

    df = df.dropna(axis=0, how='all', subset=df.columns[:-1]).reset_index().drop("index", axis = 1)

    frames = []

    for index, row in df.iterrows():

        xs = row.values[::4][:-1]
        ys = row.values[1::4][:-1]
        zs = row.values[2::4]
        vis = row.values[3::4]

        class_name = row.values[-1]
        class_id = row.values[-2]

        h1 = sum(xs[:10]) / 10
        h2 = sum(ys[:10]) / 10
        h3 = sum(zs[:10]) / 10

        xsf = [xs[m] for m in SELECTED_POINTS]
        ysf = [ys[m] for m in SELECTED_POINTS]
        zsf = [zs[m] for m in SELECTED_POINTS]

        angles = []

        for ang in ANGLES:
            angles.append(angle_between_lines(
                            xs[ang[0]], ys[ang[0]], zs[ang[0]],
                            xs[ang[1]], ys[ang[1]], zs[ang[1]],
                            xs[ang[2]], ys[ang[2]], zs[ang[2]]
                        ))
            
        sequence = []

        for x,y,z in zip(xsf, ysf, zsf):
            sequence.extend([x, y, z])
        
        frames.append(angles + [h1, h2, h3] + sequence)

    adf = pd.DataFrame(frames)
    # adf["49"] = df["132"]
    adf['class'] = df['class']

    adf.to_csv(f"aug_output/{df['class'][0]}.csv")

In [17]:
df = pd.read_csv("aug_output\\barbell biceps curl.csv").drop("Unnamed: 0", axis = 1)

In [19]:
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,class
0,0.074972,0.204167,0.209570,0.303638,0.241994,0.261190,0.333765,0.337371,0.477374,0.438145,...,0.480681,0.849860,0.057113,0.571037,1.041445,0.112750,0.495998,0.996259,0.271589,barbell biceps curl
1,0.102421,0.268143,0.208465,0.299998,0.283491,0.267813,0.369101,0.342646,0.466689,0.428334,...,0.480715,0.859802,0.110337,0.571047,1.066567,0.083413,0.496321,1.005756,0.343622,barbell biceps curl
2,0.103946,0.272242,0.209439,0.303611,0.280193,0.268588,0.366254,0.335609,0.469786,0.426333,...,0.480709,0.862233,0.103941,0.571048,1.066449,0.091319,0.496329,1.008875,0.331799,barbell biceps curl
3,0.082057,0.247695,0.209025,0.303845,0.272181,0.272681,0.355501,0.330630,0.475324,0.421904,...,0.480748,0.866243,0.092910,0.570617,1.072425,0.090237,0.496428,1.011306,0.324774,barbell biceps curl
4,0.078066,0.241757,0.206151,0.303446,0.275066,0.274123,0.355506,0.329073,0.471957,0.420963,...,0.480795,0.868692,0.097914,0.569838,1.074468,0.091299,0.496433,1.012081,0.339190,barbell biceps curl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4206,0.078296,0.037838,0.155761,0.161896,0.206580,0.341598,0.278889,0.158907,0.436987,0.365176,...,0.561035,0.963435,0.002986,0.569880,0.981618,0.212824,0.555124,1.069043,0.233369,barbell biceps curl
4207,0.079196,0.193490,0.139420,0.122463,0.206752,0.338987,0.288128,0.165133,0.454856,0.363055,...,0.562000,0.963533,0.003914,0.569251,0.988447,0.198130,0.555460,1.078924,0.221673,barbell biceps curl
4208,0.117332,0.384561,0.238725,0.310342,0.225556,0.290672,0.225947,0.192102,0.398288,0.350903,...,0.553577,0.898188,0.010402,0.572060,0.896393,0.230862,0.543235,0.898869,0.328512,barbell biceps curl
4209,0.116257,0.404574,0.210557,0.211261,0.202778,0.297046,0.247835,0.219606,0.374762,0.331176,...,0.555152,0.898176,-0.014570,0.563346,0.966670,0.237901,0.542418,0.969200,0.275709,barbell biceps curl
